# Collaborative Filtering

How do companies such as Amazon and Netflix choose what to recommend to users out of thousands of available products? A key technique they employ is *collaborative filtering*, which is the use of information from similar users and items to predict preference for a given item.

Suppose we have some movie-streaming data:

In [ ]:
import pandas as pd

#Data
raw_data = [
            [1,1,1,0,0,0],    
            [2,1,1,0,0,1],
            [3,1,0,0,0,0],
            [4,1,0,0,0,1],
            [5,0,0,1,0,1],
            [6,0,1,0,1,0],
            ]
labels = ['customer','movie1','movie2','movie3','movie4','movie5']
data = pd.DataFrame.from_records(raw_data,columns=labels)
data

Choice data such as the one above is called *implicit data*, because it only reflects the users' preference implicitly through their choices. Because it is possible that some users dislike some choices they have made, a chosen item is not the same as a prefered item. But since it is highly unlikely that most users hate most of the choices they have made, the data is usually quite informative as a whole. 

Data that includes actual preference is called *explicit data*. For example, the data might contain user ratings from 1 to 5:

In [ ]:
import pandas as pd

#Data
raw_data_explicit = [
            [1,4,2,0,0,0],    
            [2,3,2,0,0,3],
            [3,3,0,0,0,0],
            [4,5,0,0,0,4],
            [5,0,0,3,0,1],
            [6,0,3,0,4,0],
            ]
labels = ['customer','movie1','movie2','movie3','movie4','movie5']
data_explicit = pd.DataFrame.from_records(raw_data_explicit,columns=labels)
data_explicit

It is also possible that both explicit and implicit data are available. This is often the case because only a minority of users who have chosen an item will voluntarily provide ratings.

We will mostly work with implicit data, but the techniques we cover do work with both types of data in general.

## A. Nearest Neighbor

One intuitively appealing approach is to look for other users that have similar records and see what they have chosen before. 

In the following example, we will look for the three closest neighbors to a new user and average their choices:

In [ ]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

#Extract data from dataframe
X = np.asarray(data[["movie1","movie2","movie3","movie4","movie5"]])

#Only consider the three nearest neighbor


#New user who have only watched movie 1


#Get neighbor and calculate average choices


Our model suggests that we should recommend the user to try out movie 2 and movie 5. 

We can also use ```np.argsort()``` to get a list of index to recommend starting from the most recommended item:

In [ ]:
np.argsort(-(np.mean(X[neigh],axis=1) - x))

The above recommendation is based on the fact that the new user having already chosen movie 1. What if the new user has not yet tried anything? In that case, we might want to simply recommend the average of all users. In other words, we recommend the most popular items.

## B. Matrix Factorization

Matrix factorization assumes that choice data can be represented by a matrix multiplication of item features (usually called *factors*) and user preference on those features:

<img src="http://danielnee.com/wp-content/uploads/2015/04/Matrix-Factorisation-1.png">

Source: <a href="http://danielnee.com/2016/09/collaborative-filtering-using-alternating-least-squares/">Daniel Nee</a>

Let $P$ be a matrix of user preference of shape $\text{no. of users} \times \text{no. of factors}$ and $Q$ a matrix of item factors of shape $\text{no. of items} \times \text{no. of factors}$. Then the choice data $X$ can be represented by
$$
X = PQ^T
$$

There are several ways to get from $X$ to $P$ and $Q$. 

### B1. Single Value Decomposition (SVD)

Single value decomposition performs the following factorization:

$$
X = U \Sigma V^T
$$

where $\Sigma$ is a diagonal matrix of *singular values*. To get $X=PQ^T$, we can let $P=U$ and $Q^T=\Sigma V^T$.

SVD can be performed by calling ```scipy.sparse.linalg.svds(X,k)```, where $1\leq k<\min{\{\text{X.shape}\}}$ is the number of singular values. Several things to note:
- ```svds()``` expects float-point numbers, so if the data contains integers we will need to convert them to float via ```.astype(float)```.
- ```svds()``` does not return a diagonal matrix $\Sigma$, instead it returns its diagonal values in a 1-D array. We can apply ```np.diag()``` on this array to get $\Sigma$. 

Let us try running ```svds``` on our data:

In [ ]:
import numpy as np
from scipy.sparse import csc_matrix
from scipy.sparse.linalg import svds



The $P$ matrix represents user preference. Each row is one user and each column is the user's preference for a particular factor:

In [ ]:
print(P)

Since the factors are automatically generated, it is hard to know what they represent without further investigation. For movies, you can imagine that one factor might represent how much action element there is, while another might represent how much romance element there is. In any case, from the perspective of generating recommendation we do not necessarily care about what these underlying factors are, since what we want is $PQ^T$.

Now let us take a look at $Q^T$:

In [ ]:
#Compute Qt


#print out the content of Qt
print(Qt)

Each column is one item and each row is the item's exposure to a particular factor.

What is the effect of having different values of $k$? Let's try that out:

In [ ]:
#k=4


print("k =",k)
print(np.round(np.dot(P,Qt),5))

#k=2


print("k =",k)
print(np.round(np.dot(P,Qt),5))


Note how much more $PQ^T$ resembles $X$ when $k$ is large. Does that mean we want a large $k$ then? Far from it. A large $k$ means that our model will predict the existing chocies of the users perfectly. If the user has not chosen an item before, the model will predict that she does not like the item, resulting in no recommendation. In other words, our model is overfitting the data. 

As you should now realize, $k$ is the main hyperparameter that you would want to tune in SVD models.

How do we generate recommendations for an arbitrary user, particular one that is not already in the data? Since

$$
X = PQ^T \\
XQ = PQ^TQ \\
XQ(Q^TQ)^{-1} = P
$$

This gives us an equation to find the preference of a particular user $u$:

$$
p_u = x_uQ(Q^TQ)^{-1}
$$

Which in turn allows us to predict what the user might choose:

$$
\hat{x}_u = p_uQ^T
$$

Let $S=Q(Q^TQ)^{-1}$. Then $p_u = x_uS$:

In [ ]:
#New user who have only watched movie 1
x = np.array([1,0,0,0,0])

#Generate prediction


As in the case of nearest neighbor, our model suggests that we should recommend the user to try out movie 2 and movie 5.

We will write a simple class that implements an interface similar to ```scikit-learn```. We will call it ```SVDarnoldi``` because ```svds``` implements the <a href="https://en.wikipedia.org/wiki/Arnoldi_iteration#Implicitly_restarted_Arnoldi_method_.28IRAM.29">Arnoldi iteration</a>.  

The class needs to expose two methods: 
- ```fit(X)``` that fits the model for a given set of training data ```X```.
- ```predict(x)``` that generates prediction for a given set of new data ```x```.

In [ ]:
#Implement SVDarnoldi here


With this class, it is easy to try out different values of $k$:

In [ ]:
#New user who have only watched movie 1
x = np.array([1,0,0,0,0])

#Loop through
for k in range(1,5):
    print("k =",k)
    model = SVDarnoldi(k=k)
    model.fit(X)
    print(model.predict(x))

### B2. Alternating Least Squared (ALS)

For efficiency reasons, practical implementations of matrix-factorization-based filtering mostly utilize approximation of SVD. Here we will cover a method called *alternating least squared*. The idea is as follows: we first randomly initiatizes $P$ and $Q^T$, and then iteratively update these two matrices until $PQ^T \approx X$. But how should we update the matrices? 

For clarity, I will be using $'$ instead of $^T$ to denote inverse. Starting with the objective $PQ' = X$:

$$
PQ' = X \\
PQ'Q = XQ \\
P = XQ(Q'Q)^{-1}
$$

Similarly, 

$$
PQ' = X  \\
P'PQ' = P'X \\
Q' = (P'P)^{-1}P'X 
$$

So what we are going to do is to run $P = XQ(Q'Q)^{-1}$ and $Q = (P'P)^{-1}P'X$ iteratively until $PQ' \approx X$. 

Note that what we are doing here are essentially running OLS repeatedly until we converge on a stable combination of $P$ and $Q'.$ Now it is true that when we run OLS we usually have a single column vector of dependent variable, whereas here $X$ is a matrix, but the technique is the same.

Reference:
- <a href="http://citeseerx.ist.psu.edu/viewdoc/download;jsessionid=34AEEE06F0C2428083376C26C71D7CFF?doi=10.1.1.167.5120&rep=rep1&type=pdf">Collaborative Filtering for Implicit Datasets</a>
- <a href="https://pdfs.semanticscholar.org/dbe9/d04bffb5c1df8eb721dab4f744ea81d9a4c1.pdf">Alternating Least Squared for Personalized Ranking</a>

Below is a straight-forward implementation of ALS:

In [ ]:
#Implement ALS here


The predicted preference is essentially the same as what we got from a real SVD.

If we are going use the algorithm repeatedly, however, it would be best to write a self-contained class that we can use repeatedly:

In [ ]:
class SVDals():
    """
    Alternating Least Square SVD
    """    
        
    def __init__(self,k=2,min_loss_delta=0.00001,max_epochs=20):
        """
        k:               Number of latent factors
        min_loss_delta:  Minimum change in mean squared error to continue training
        max_epochs:      Maximum number of training rounds
        """
        self.k = k
        self.min_loss_delta = min_loss_delta
        self.max_epochs = max_epochs
       
    def fit(self,X):
        """
        Fit the model
        X: training data
        """
        
        #Initialize model parameters
        self.initialize(X)
        loss, loss_delta = self.update_loss(X,0) 
        
        print("Training...")
        epoch = 0
        while loss_delta > self.min_loss_delta and epoch < self.max_epochs:
            #Update parameters
            self.update_params()
            
            #Update error and loss
            loss, loss_delta = self.update_loss(X,loss)
            
            #Increment counter
            epoch = epoch + 1

            #Show each round's epoch and self.error
            self._printloss(epoch,loss)
       
    def initialize(self,X):
        """
        Initializes P and Qt
        """
        self.user_count = X.shape[0]
        self.item_count = X.shape[1]
        #P and Qt uniformly distributed from -0.5 to 0.5
        self.P = np.random.rand(self.user_count,self.k) - 0.5 
        self.Qt = np.random.rand(self.k,self.item_count ) - 0.5
        
    def update_params(self):
        """
        Update parameters
        """        
        #Update P and Qt
        self.P = np.dot(X,np.dot(self.Qt.T,np.linalg.inv(np.dot(self.Qt,self.Qt.T))))
        self.Qt = np.dot(np.dot(np.linalg.inv(np.dot(self.P.T,self.P)),self.P.T),X)
        self.S = np.dot(self.Qt.T,np.linalg.inv(np.dot(self.Qt,self.Qt.T)))
                
    def update_loss(self,X,loss_prev):
        """
        Update self.error and mean squared self.error
        """        
        #Generate Prediction   
        X_hat = np.dot(self.P,self.Qt)
        
        #Error matrix
        self.error = X - X_hat
        
        #loss
        loss = self._loss(X,X_hat)
        loss_delta = abs(loss_prev - loss)  
        
        return loss, loss_delta  
            
    def _loss(self,X,X_hat):
        """
        Calculate mean squared error
        """
        return np.mean(np.asarray(np.square(X - X_hat)))
    
    def _printloss(self,epoch,loss):
        """
        Print formated loss
        """
        print(str(epoch).ljust(3),"loss:",round(loss,4))
        
    def predict(self,x):
        """
        Inference
        x: input data
        """        
        p = np.dot(x,self.S)
        x_hat = np.dot(p,self.Qt)
        return x_hat

Let us try it out:

In [ ]:
model = SVDals(k=2)
model.fit(X)
print(model.predict(x))

The main advantage of ALS is that it is highly parallelizable and converges very quickly, resulting in very fast training. This is in contrast to the stochastic gradient descent approach that we will cover next.

### B3. Gradient Descent

*Gradient descent* nudges parameters by an amount proportional to their contribution to the loss function. Gradient Descent and its approximation, *Stochastic Gradient Descent* (SGD), are very general optimization methods, usable in all sorts of models from logistic regression to neural network.

A simple example is as follows: Suppose our model is

$$
\hat{y} = \alpha + x
$$

As is common in regression problem, we would like to minimize the squared error. So our loss function is:

$$
c = \left( y - \hat{y} \right)^2
$$

<img src="http://www.ticoneva.com/econ/econ4130/images/loss-error.png" width="300">

We have an initial guess of what $\alpha$ is---often just a random number---and an initial prediction $\hat{y}_0 = \alpha_0 + x$. This prediction is likely inaccurate, which means the loss will be positive:

$$
c_0 = \left( y - \hat{y}_0 \right)^2 > 0
$$

How do we use this information to update $\alpha$? Let $\epsilon_0 = y - \hat{y}_0$. 
The marginal effect, or *gradient*, of $\alpha$ on $c$ is:

$$
\frac{\partial c}{\partial \alpha} 
= \frac{\partial}{\partial \alpha}\left( y-\hat{y} \right)^2 = -2 \epsilon
$$

Suppose the error is positive, so $\epsilon_0 = y - \hat{y}_0  > 0$. The loss function will be decreasing in $\alpha$, which makes sense---$\hat{y}$ is increasing in $\alpha$, and right now $\hat{y} < y$. We can make our model more accurate by increasing $\alpha$. Conversely, we should decrease $\alpha$ if the error is negative.

The gradient thus tells us the direction we need to adjustment our parameter. Furthermore, the amount we need to adjust is, to a first-order approximation, proportional to

$$
- \frac{\partial c}{\partial \alpha}
$$

We therefore have the following update rule:

$$
\alpha_{t} = \alpha_{t-1} - \gamma \frac{\partial c}{\partial \alpha} \bigg\rvert_{\alpha_{t-1}}
$$

Or more typical in computer science:

$$
\alpha \gets \alpha - \gamma \frac{\partial c}{\partial \alpha} 
$$

$\gamma$ is called the *learning rate*. Learning rate is usually much smaller than 1 to prevent overshooting. It can be manually specified in simple settings such as ours but is often automatically adjusted in more advance alogrithms.

There are a couple of options when it comes to the computation of gradient:
- Averaging the gradient from all samples. The advantage of this method is that the "true" gradient is used, in the sense that it reflects the overall gradient of the training data. The disadvantage is that the speed of convergence is slow, since we are only updating the model parameters after we compute the gradient of all observations.
- **Stochastic Gradient Descent (SGD):** Update parameters with the average gradient of a subset of samples. Some updates will push the parameters in one direction while some others will push them in the other direction---this is the *stochastic* part of the algorithm---but on average the parameters will move towards the right direction. Besides allowing for faster convergence, the fact that the gradient is noisy also helps the model avoid local minimas. 

Large-scale machine learning models are typically trained on variations of SGD. Data is broken into mutually-exclusive groups called *mini-batches* and model parameters are updated with the average gradient of each mini-batch.  

Now specifically for SVD, we have for each user $u$ and each item $i$,

$$
\hat{x}_{ui} = p_u q^T_i
$$

The loss function is:

$$
\sum_{u,i}{\left( x_{ui} - \hat{x}_{ui} \right)^2}
$$

so the gradient consists of:

$$
\frac{\partial c_{ui}}{\partial p_u} = -2 \epsilon_{ui} \cdot q_i \\
\frac{\partial c_{ui}}{\partial q_i} = -2 \epsilon_{ui} \cdot p_u
$$

The update rules for $P$ and $Q^T$ are thus:

$$
P \gets P + \gamma \mathcal{E} Q \\
Q^T \gets Q^T + \gamma P^T \mathcal{E}
$$

As before, here is a straight-forward implementation of gradient descent SVD:

In [ ]:
#Implement GD-SVD here


Note how many more epochs it takes for gradient descent to converge in contrast to ALS. Can we speed things up by setting a higher learning rate? If we try different learning rates, we will see that having too high a learning rate would result in constant overshooting, and as a result no convergence.

Below is a class implementing gradient descent SVD:

In [ ]:
class SVDgd():
    """
    Gradient Descent SVD
    """
        
    def __init__(self,k=2,
                 min_loss_delta=0.00001,
                 max_epochs=100,
                 learning_rate=0.1,
                 show_progress=True
                ):
        """
        k:               Number of latent factors
        min_loss_delta:   Minimum change in mean squared error to continue training
        max_epochs:      Maximum number of training rounds
        learning_rate:   Learning rate
        show_progress:   Print error of each epoch
        """
        self.k = k
        self.min_loss_delta = min_loss_delta
        self.max_epochs = max_epochs
        self.learning_rate = learning_rate
        self.show_progress = show_progress
        
    def fit(self,X):
        """
        Fit the  model
        X: training data
        """
        
        #Initialize model parameters
        self.initialize(X)
        loss, loss_delta = self.update_loss(X,0) 
        
        print("Training...")
        epoch = 0
        while loss_delta > self.min_loss_delta and epoch < self.max_epochs:
            #Update parameters
            self.update_params()
            
            #Update error and loss
            loss, loss_delta = self.update_loss(X,loss)
            
            #Increment counter
            epoch = epoch + 1
            
            if self.show_progress:
                #Show each round's epoch and self.error
                self._printloss(epoch,loss)
                
        if not self.show_progress:
            #Show the final epoch and self.error
            self._printloss(epoch,loss)
       
    def initialize(self,X):
        """
        Initializes P and Qt
        """
        self.user_count = X.shape[0]
        self.item_count = X.shape[1]
        #P and Qt uniformly distributed from -0.5 to 0.5
        self.P = np.random.rand(self.user_count,self.k) - 0.5 
        self.Qt = np.random.rand(self.k,self.item_count ) - 0.5
        
    def update_params(self):
        """
        Update parameters
        """        
        #Update P and Qt with previous epoch's loss
        self.P = self.P + self.learning_rate * (np.dot(self.error, self.Qt.T))
        self.Qt = self.Qt + self.learning_rate * (np.dot(self.P.T,self.error))
        self.S = np.dot(self.Qt.T,np.linalg.inv(np.dot(self.Qt,self.Qt.T)))
                
    def update_loss(self,X,loss_prev):
        """
        Update self.error and mean squared self.error
        """        
        #Generate Prediction   
        X_hat = np.dot(self.P,self.Qt)
        
        #Error matrix
        self.error = X - X_hat
        
        #loss
        loss = self._loss(X,X_hat)
        loss_delta = abs(loss_prev - loss)  
        
        return loss, loss_delta  
            
    def _loss(self,X,X_hat):
        """
        Calculate mean squared error
        """
        return np.mean(np.asarray(np.square(X - X_hat)))
    
    def _printloss(self,epoch,loss):
        """
        Print formated loss
        """
        print(str(epoch).ljust(3),"loss:",round(loss,4))
        
    def predict(self,x):
        """
        Inference
        x: input data
        """        
        p = np.dot(x,self.S)
        x_hat = np.dot(p,self.Qt)
        return x_hat

As before, let us try out the algorithm:

In [ ]:
model = SVDgd(k=2)
model.fit(X)
print(model.predict(x))

### B3. Simon Funk's SVD

This is a method popularized during the Netflix Prize, and it is usually what people refers to when they mention "SVD" in the context of collaborative filtering.  

The prediction of the model is given by:
$$
\hat{x}_{ui} = \mu + b_u + b_i + p_u q^T_i
$$

$\mu$, $b_u$ and $b_i$ are called *bias* in machine learning, but a more familiar name for economists would be dummy variables. So Funk's SVD is essentially SVD with fixed effects.

The model is also regularized, so the loss function is:
$$
\sum_{u,i}{\left( x_{ui} - \hat{x}_{ui} \right)^2 
+ \alpha \left( b_u^2 + b_i^2 + \lVert p_u \rVert^2 + \lVert q_i \rVert^2  \right) }
$$
where $\alpha$ is the strength of regularization. 

Reference:
- <a href="http://sifter.org/~simon/journal/20061211.html">Netflix Update: Try This at Home</a>
 
Below we extend the ```SVDsgd``` class to create a new ```SVDfunk``` class. Since we have more parameters to estimate we will increase the maximum epochs to give the model more time to train.

In [ ]:
class SVDfunk(SVDgd):
    """
    Simon Funk's SVD. Regularized.
    """
    
    def __init__(self,alpha=0,max_epochs=500,*args,**kargs):
        """
        alpha: regularization strength
        """    
        self.alpha = alpha
        #pass other arguments to parent class
        super().__init__(max_epochs=max_epochs,*args,**kargs)
      
    def initialize(self,X):
        """
        Initializes biases, P and Qt
        """
        super().initialize(X) #Use parent class to initialize P and Qt
        self.mu = np.mean(X)
        self.bu = np.random.normal(size=(self.user_count,1)) 
        self.bi = np.random.normal(size=(self.item_count,1))
        
    def update_params(self):
        """
        Update parameters
        """        
         #Update self.biases, self.P and self.Qt with previous epoch's loss
        self.mu = self.mu + self.learning_rate * np.mean(self.error)
        self.bu = self.bu + self.learning_rate * (
                np.mean(self.error,axis=1).reshape(self.user_count,1)
                - self.alpha * self.bu
                )
        self.bi = self.bi + self.learning_rate * (
                np.mean(self.error,axis=0).reshape(self.item_count,1)
                - self.alpha * self.bi
                )
        self.P = self.P + self.learning_rate * (
                np.dot(self.error, self.Qt.T)
                - self.alpha * self.P
                )
        self.Qt = self.Qt + self.learning_rate * (
                np.dot(self.P.T,self.error)
                - self.alpha * self.Qt
                )
        self.S = np.dot(self.Qt.T,np.linalg.inv(np.dot(self.Qt,self.Qt.T)))
        
    def update_loss(self,X,loss_prev):
        """
        Update error and mean squared self.error
        """        
        #Generate Prediction
        ones_i = np.ones((1,self.item_count))
        ones_u = np.ones((self.user_count,1))        
        X_hat = (self.mu + np.dot(self.bu,ones_i) 
                 + np.dot(ones_u,self.bi.T) 
                 + np.dot(self.P,self.Qt))  
        
        #Error matrix
        self.error = X - X_hat
        
        #Regularized loss
        loss = self._loss(X,X_hat) + self.alpha * (
                np.mean(self.bu**2)
                + np.mean(self.bi**2)
                + np.mean(self.P**2)
                + np.mean(self.Qt**2)
                )
        loss_delta = abs(loss_prev - loss)  
        
        return loss, loss_delta  

    def predict(self,x):
        """
        Inference
        x: input data
        """        
        p = np.dot(x,self.S)
        x_hat = self.mu + self.bi.T + np.dot(p,self.Qt) #user bias is zero for new user
        return x_hat    

Now let us try out the model:

In [ ]:
model = SVDfunk(k=2,alpha=0.05,show_progress=False)
model.fit(X)
print(model.predict(x))

At first sight, there seems to be little difference between ```SVDfunk``` and ```SVDsgd```. Taking a closer look at the mean-squared errors of the two models, however, and it is clear that ```SVDfunk``` performs better for this particular metric.

In [ ]:
#SGD
model = SVDgd(k=2,show_progress=False)
model.fit(X)
print(model.predict(x))

#Funk's
model = SVDfunk(k=2,show_progress=False)
model.fit(X)
print(model.predict(x))

A similar performance lead exists for explicit data:

In [ ]:
#Extract data from dataframe
X2 = np.asarray(data_explicit[["movie1","movie2","movie3","movie4","movie5"]])

#New user who rated movie1 with a 3
x2 = np.array([3,0,0,0,0])

#SGD
model = SVDgd(k=2,show_progress=False)
model.fit(X2)
print(model.predict(x2))

#Funk's
model = SVDfunk(k=2,show_progress=False)
model.fit(X2)
print(model.predict(x2))

## Conclusion

We have gone through a few different methods of generating recommendations based on existing records. 

If you are interested in collaborative filtering, be sure to check out the Netflix Prize. The data is <a href="https://www.kaggle.com/netflix-inc/netflix-prize-data">available on Kaggle</a>, and you can find the winning team's research papers <a href="https://netflixprize.com/community/topic_1537.html">here</a>. One thing you will find is the the winning teams all employ an ensemble of models, which very often perform better than any single model.